In [105]:
import pandas as pd
import aggregation

In [106]:
ewt = pd.read_excel('elections_with_tracts.xlsx')
# get rows where filename.lower() contains ward12
row = ewt[ewt['filename'] == 'SanLeandro_11062018_CountyCouncilDistrict1.csv']
row

,filename,District #,tracts,percents,num_tracts
18,SanLeandro_11062018_CountyCouncilDistrict1.csv,1,"06_001_410000, 06_001_433104, 06_001_430400, 0...","0.000662315473001776, 0.8697637238945869, 0.00...",10


In [107]:
tracts = row['tracts'].values[0].split(', ')
percents = row['percents'].values[0].split(', ')
percent_dict = dict(zip(tracts, percents))
percent_dict

{'06_001_410000': '0.000662315473001776',
 '06_001_433104': '0.8697637238945869',
 '06_001_430400': '0.0013243275122125852',
 '06_001_432800': '0.4575513767953343',
 '06_001_433103': '0.0017520949698166248',
 '06_001_432501': '0.0054157394023728565',
 '06_001_433000': '0.017511927909202776',
 '06_001_432700': '0.98800583489135',
 '06_001_432600': '0.7884982356472572',
 '06_001_432100': '0.0022974352431301905'}

In [108]:
tracts

['06_001_410000',
 '06_001_433104',
 '06_001_430400',
 '06_001_432800',
 '06_001_433103',
 '06_001_432501',
 '06_001_433000',
 '06_001_432700',
 '06_001_432600',
 '06_001_432100']

In [117]:
# open acs_data.csv
acs = pd.read_csv('tract_acs_data.csv')
# strip geocode column
acs['geocode'] = acs['geocode'].str.strip()
tract_data = acs[acs['geocode'].isin(tracts)]
# get rows where tract_data[year] is 2018
tract_data = tract_data[tract_data['year'] == 2018]
# only keep geocode, year, DP05_0001E, and DP02_0154E
tract_data = tract_data[['geocode', 'year', 'DP05_0001E', 'DP02_0154E']]
percent_df = pd.DataFrame({'percent': [percent_dict[tract] for tract in tract_data['geocode']]})
#reset index inm each
tract_data = tract_data.reset_index(drop=True)
percent_df = percent_df.reset_index(drop=True)
#concat with index
tract_data = pd.concat([tract_data, percent_df], axis=1)
# set DP05_0001E to 1861.97 in row 8
tract_data.at[8, 'DP02_0154E'] = 2361.41
tract_data

/var/folders/52/t8__xqw525g96s6z30kmvnfr0000gp/T/ipykernel_73109/864545175.py:2: DtypeWarning: Columns (13,68,69,71,72,79,105,106,122,133,135,151,171,204,208,209,218,221,224,233,238,256,262,283,287,302,320,322,334,355,361,369,370,378,394,403,419,420,421,446,447,451,457,460,464,470,475,476,505,525,532,533,539,542,551,564,574,598,607,618,631,635,648,656,657,660,673,677,684,692,693,694,703,704,729,743,756) have mixed types. Specify dtype option on import or set low_memory=False.
  acs = pd.read_csv('tract_acs_data.csv')


,geocode,year,DP05_0001E,DP02_0154E,percent
0,06_001_410000,2018,3358,1020.00,0.000662315473001776
1,06_001_433104,2018,4044,1477.00,0.8697637238945869
2,06_001_430400,2018,2108,647.00,0.0013243275122125852
3,06_001_432800,2018,3907,1256.00,0.4575513767953343
4,06_001_433103,2018,3486,1248.00,0.0017520949698166248
5,06_001_432501,2018,4850,1344.00,0.0054157394023728565
6,06_001_433000,2018,3461,1311.00,0.017511927909202776
7,06_001_432700,2018,2839,869.00,0.98800583489135
8,06_001_432600,2018,7043,2361.41,0.7884982356472572
9,06_001_432100,2018,3584,1262.00,0.0022974352431301905


In [118]:
# add column that is DP02_0154E / DP05_0001E called proportion
tract_data['proportion'] = tract_data['DP02_0154E'] / tract_data['DP05_0001E']
tract_data

,geocode,year,DP05_0001E,DP02_0154E,percent,proportion
0,06_001_410000,2018,3358,1020.00,0.000662315473001776,0.303752
1,06_001_433104,2018,4044,1477.00,0.8697637238945869,0.365232
2,06_001_430400,2018,2108,647.00,0.0013243275122125852,0.306926
3,06_001_432800,2018,3907,1256.00,0.4575513767953343,0.321474
4,06_001_433103,2018,3486,1248.00,0.0017520949698166248,0.358003
5,06_001_432501,2018,4850,1344.00,0.0054157394023728565,0.277113
6,06_001_433000,2018,3461,1311.00,0.017511927909202776,0.378792
7,06_001_432700,2018,2839,869.00,0.98800583489135,0.306094
8,06_001_432600,2018,7043,2361.41,0.7884982356472572,0.335285
9,06_001_432100,2018,3584,1262.00,0.0022974352431301905,0.352121


In [119]:
# add tract_pop that is percent times DP05_0001E after converting to floats
tract_data['tract_pop'] = tract_data['percent'].astype(float) * tract_data['DP05_0001E'].astype(float)
tract_data

,geocode,year,DP05_0001E,DP02_0154E,percent,proportion,tract_pop
0,06_001_410000,2018,3358,1020.00,0.000662315473001776,0.303752,2.224055
1,06_001_433104,2018,4044,1477.00,0.8697637238945869,0.365232,3517.324499
2,06_001_430400,2018,2108,647.00,0.0013243275122125852,0.306926,2.791682
3,06_001_432800,2018,3907,1256.00,0.4575513767953343,0.321474,1787.653229
4,06_001_433103,2018,3486,1248.00,0.0017520949698166248,0.358003,6.107803
5,06_001_432501,2018,4850,1344.00,0.0054157394023728565,0.277113,26.266336
6,06_001_433000,2018,3461,1311.00,0.017511927909202776,0.378792,60.608782
7,06_001_432700,2018,2839,869.00,0.98800583489135,0.306094,2804.948565
8,06_001_432600,2018,7043,2361.41,0.7884982356472572,0.335285,5553.393074
9,06_001_432100,2018,3584,1262.00,0.0022974352431301905,0.352121,8.234008


In [120]:
# do weighted summ of DP02_0154E with percent as weight
weighted_sum = (tract_data['DP02_0154E'] * tract_data['percent'].astype(float)).sum()
weighted_sum

4616.725509335396

In [80]:
# weighted_prop times tract_pop in row of index 8 of tract_data
prediction = weighted_prop * tract_data['tract_pop'][8]
prediction

1861.9714470161434

In [101]:
# open statistics.xlsx
stats = pd.read_excel('statistics.xlsx')
# get the second row of each 'type'
stats = stats.groupby('type').last()
stats
# open variables_acs5_2022
variables = pd.read_csv(r'variables/variables_acs5_2022.csv')

# for each row in stats add a column Label with the label from variables where Name is agg_var and a column Population Label where Name is pop_vae
stats['Label'] = [variables[variables['Name'] == row['agg_var']]['Label'].values[0] for index, row in stats.iterrows()]
stats['Population Label'] = [variables[variables['Name'] == row['pop_var']]['Label'].values[0] for index, row in stats.iterrows()]
stats

stats

,agg_var,pop_var,Label,Population Label
type,,,,
mean,DP03_0091E,DP03_0089E,Estimate!!INCOME AND BENEFITS (IN 2022 INFLATI...,Estimate!!INCOME AND BENEFITS (IN 2022 INFLATI...
median,DP05_0018E,DP05_0001E,Estimate!!SEX AND AGE!!Total population!!Media...,Estimate!!SEX AND AGE!!Total population
percentage,DP03_0009PE,DP03_0008E,Estimate!!EMPLOYMENT STATUS!!Civilian labor fo...,Estimate!!EMPLOYMENT STATUS!!Civilian labor force
ratio,DP05_0032E,DP05_0031E,Estimate!!SEX AND AGE!!Total population!!65 ye...,Estimate!!SEX AND AGE!!Total population!!65 ye...


In [102]:
# print each label
for label in stats['Label']:
    print(label)

Estimate!!INCOME AND BENEFITS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Nonfamily households!!Mean nonfamily income (dollars)
Estimate!!SEX AND AGE!!Total population!!Median age (years)
Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate
Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females)


In [103]:
# print each popualtiln label
for label in stats['Population Label']:
    print(label)

Estimate!!INCOME AND BENEFITS (IN 2022 INFLATION-ADJUSTED DOLLARS)!!Nonfamily households
Estimate!!SEX AND AGE!!Total population
Estimate!!EMPLOYMENT STATUS!!Civilian labor force
Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female
